## Preprocessing a few things before importing the data

In [3]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)

from pyspark.sql.functions import *

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_3662/2158267922.py:2 

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark - example toPandas()').getOrCreate()

In [8]:
df = spark.read.option("header", "true").csv('vaccination.csv')

In [9]:
df.count()

164015

In [10]:
df.columns

['iso_code', 'continent', 'location', 'date']

In [11]:
df.columns

['iso_code', 'continent', 'location', 'date']

In [12]:
df.dtypes

[('iso_code', 'string'),
 ('continent', 'string'),
 ('location', 'string'),
 ('date', 'string')]

In [13]:
df.select(col('date')).show()

+---------+
|     date|
+---------+
|2/24/2020|
|2/25/2020|
|2/26/2020|
|2/27/2020|
|2/28/2020|
|2/29/2020|
| 3/1/2020|
| 3/2/2020|
| 3/3/2020|
| 3/4/2020|
| 3/5/2020|
| 3/6/2020|
| 3/7/2020|
| 3/8/2020|
| 3/9/2020|
|3/10/2020|
|3/11/2020|
|3/12/2020|
|3/13/2020|
|3/14/2020|
+---------+
only showing top 20 rows



In [14]:
df.filter(col("date").like("_/__/____")).show()

+--------+---------+-----------+---------+
|iso_code|continent|   location|     date|
+--------+---------+-----------+---------+
|     AFG|     Asia|Afghanistan|2/24/2020|
|     AFG|     Asia|Afghanistan|2/25/2020|
|     AFG|     Asia|Afghanistan|2/26/2020|
|     AFG|     Asia|Afghanistan|2/27/2020|
|     AFG|     Asia|Afghanistan|2/28/2020|
|     AFG|     Asia|Afghanistan|2/29/2020|
|     AFG|     Asia|Afghanistan|3/10/2020|
|     AFG|     Asia|Afghanistan|3/11/2020|
|     AFG|     Asia|Afghanistan|3/12/2020|
|     AFG|     Asia|Afghanistan|3/13/2020|
|     AFG|     Asia|Afghanistan|3/14/2020|
|     AFG|     Asia|Afghanistan|3/15/2020|
|     AFG|     Asia|Afghanistan|3/16/2020|
|     AFG|     Asia|Afghanistan|3/17/2020|
|     AFG|     Asia|Afghanistan|3/18/2020|
|     AFG|     Asia|Afghanistan|3/19/2020|
|     AFG|     Asia|Afghanistan|3/20/2020|
|     AFG|     Asia|Afghanistan|3/21/2020|
|     AFG|     Asia|Afghanistan|3/22/2020|
|     AFG|     Asia|Afghanistan|3/23/2020|
+--------+-

In [15]:
df = df.withColumn('date',when(df.date.like("_/__/____"),concat(lit("0"),df.date))
          .when(df.date.like("_/_/____"),concat(lit("0"),df.date.substr(1,2),lit("0"),df.date.substr(3,1),df.date.substr(4,5)))
          .when(df.date.like("__/_/____"),concat(df.date.substr(1,3),lit("0"),df.date.substr(4,1),df.date.substr(5,5)))
          .otherwise(df.date))


In [16]:
df.show(10)

+--------+---------+-----------+----------+
|iso_code|continent|   location|      date|
+--------+---------+-----------+----------+
|     AFG|     Asia|Afghanistan|02/24/2020|
|     AFG|     Asia|Afghanistan|02/25/2020|
|     AFG|     Asia|Afghanistan|02/26/2020|
|     AFG|     Asia|Afghanistan|02/27/2020|
|     AFG|     Asia|Afghanistan|02/28/2020|
|     AFG|     Asia|Afghanistan|02/29/2020|
|     AFG|     Asia|Afghanistan|03/01/2020|
|     AFG|     Asia|Afghanistan|03/02/2020|
|     AFG|     Asia|Afghanistan|03/03/2020|
|     AFG|     Asia|Afghanistan|03/04/2020|
+--------+---------+-----------+----------+
only showing top 10 rows



In [17]:
from pyspark.sql.functions import regexp_replace
df = df.withColumn('date', regexp_replace('date', '/', '-')) \
#.show(truncate=False)

In [18]:
df.show()

+--------+---------+-----------+----------+
|iso_code|continent|   location|      date|
+--------+---------+-----------+----------+
|     AFG|     Asia|Afghanistan|02-24-2020|
|     AFG|     Asia|Afghanistan|02-25-2020|
|     AFG|     Asia|Afghanistan|02-26-2020|
|     AFG|     Asia|Afghanistan|02-27-2020|
|     AFG|     Asia|Afghanistan|02-28-2020|
|     AFG|     Asia|Afghanistan|02-29-2020|
|     AFG|     Asia|Afghanistan|03-01-2020|
|     AFG|     Asia|Afghanistan|03-02-2020|
|     AFG|     Asia|Afghanistan|03-03-2020|
|     AFG|     Asia|Afghanistan|03-04-2020|
|     AFG|     Asia|Afghanistan|03-05-2020|
|     AFG|     Asia|Afghanistan|03-06-2020|
|     AFG|     Asia|Afghanistan|03-07-2020|
|     AFG|     Asia|Afghanistan|03-08-2020|
|     AFG|     Asia|Afghanistan|03-09-2020|
|     AFG|     Asia|Afghanistan|03-10-2020|
|     AFG|     Asia|Afghanistan|03-11-2020|
|     AFG|     Asia|Afghanistan|03-12-2020|
|     AFG|     Asia|Afghanistan|03-13-2020|
|     AFG|     Asia|Afghanistan|

In [19]:
df = df.withColumn(("date"),to_date(col("date"),"MM-dd-yyyy"))

In [20]:
df.show()

+--------+---------+-----------+----------+
|iso_code|continent|   location|      date|
+--------+---------+-----------+----------+
|     AFG|     Asia|Afghanistan|2020-02-24|
|     AFG|     Asia|Afghanistan|2020-02-25|
|     AFG|     Asia|Afghanistan|2020-02-26|
|     AFG|     Asia|Afghanistan|2020-02-27|
|     AFG|     Asia|Afghanistan|2020-02-28|
|     AFG|     Asia|Afghanistan|2020-02-29|
|     AFG|     Asia|Afghanistan|2020-03-01|
|     AFG|     Asia|Afghanistan|2020-03-02|
|     AFG|     Asia|Afghanistan|2020-03-03|
|     AFG|     Asia|Afghanistan|2020-03-04|
|     AFG|     Asia|Afghanistan|2020-03-05|
|     AFG|     Asia|Afghanistan|2020-03-06|
|     AFG|     Asia|Afghanistan|2020-03-07|
|     AFG|     Asia|Afghanistan|2020-03-08|
|     AFG|     Asia|Afghanistan|2020-03-09|
|     AFG|     Asia|Afghanistan|2020-03-10|
|     AFG|     Asia|Afghanistan|2020-03-11|
|     AFG|     Asia|Afghanistan|2020-03-12|
|     AFG|     Asia|Afghanistan|2020-03-13|
|     AFG|     Asia|Afghanistan|

In [23]:
df.toPandas().to_csv("clean_covid.csv",index = False)

In [364]:
#Since this is a small dataset, there is no need to use spark to export the csv
#df.write.csv("covid_sql.csv")